In [153]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from pathlib import Path

from core.utils import Picture

In [154]:
# Setting up I/O
input_dir = Path.cwd() / "data" / "raw"
output_dir = Path.cwd() / "data" / "output"

choose_pic = Picture.TREES

img = mpimg.imread(f"{input_dir}/{choose_pic.value}")

In [155]:
# Validating Input
print(type(img))
print(img.shape)
print(img[4,4])

<class 'numpy.ndarray'>
(2032, 3596, 4)
[0. 0. 0. 1.]


In [156]:
# Tools
rng = np.random
height, width, channels = img.shape


In [157]:
# image channelling
# red = img.copy()
# green = img.copy()
# blue = img.copy()

# red = img[:, :, 0]
# green = img[:, :, 1]
# blue = img[:, :, 2]
# channels = {
#     "red": red,
#     "green": green,
#     "blue": blue
# }

In [158]:
# new_image = img.copy()
# new_title = "Impulse Response Analysis" # IRA
# ira = rng.random_integers(0, 255, size=(height, width, channels))
# ira_mixed = (new_image.astype(float) + ira.astype(float)) / 2
# new_image[:] = ira_mixed.astype('uint8')
# # or
# new_image[:,::2] = ira[:, ::2]


In [159]:
# new_image = img.copy()
# new_title = "Эффект ненастроеного телевизора"
# new_image[:,:] = rng.random_integers(0, 255, size=(height, width, channels))

In [160]:
# new_image = img.copy()
# new_title = "10% noise"
# new_image[rng.random(size=(height, width)) < 0.1] = rng.random_integers(0, 256, (3,))

In [161]:
# We will try to flip the image horizontally
# new_image = img.copy()
# new_title = "horizontally flipped"
# new_image = new_image[:,::-1]

In [162]:
# We will try vertically flipped
# new_image = img.copy()
# new_title = "vertically flipped"
# new_image = new_image[::-1, :]

In [163]:
# We will try both flippes
# new_image = img.copy()
# new_title = "Horizontally and Verticallu flipped"
# new_image = new_image[::-1, ::-1]

In [164]:
# We will try crop center
# new_image = img.copy()
# new_title = "Cropped Image"
# min_y, max_y = np.round(height * 0.2).astype(int), np.round(height * 0.8).astype(int)
# min_x, max_x = np.round(width * 0.2).astype(int), np.round(width * 0.8).astype(int)
# new_image = new_image[min_y: max_y, min_x: max_x]

In [165]:
# We will try invert color
# new_image = img.copy()
# new_title = "Inverted Colors"
# new_image[:, :, :3] = 1.0 - new_image[:, :, :3]


# Forward mapping
1. We get original coord [y,x]
2. Calculate center, c_y, c_xw = height/2, width/2
3. Move original coord to center, moved_coord = orig[y,x] - [c_y, c_x]
4. Use following formulas to calculate turned coord
    x' = xcosa - ysina
    y' = xsina + ycosa
5. Move turned coord back to his place and get new coord for original coord:
    new_x = x' + c_x
    new_y = y' + c_y

# Inverse mapping formule
x = x'cosa + y'sina
y = -x'sina + y'cosa 

x' = xcosa - ysina 
y' = xsina + ycosa

new_x = x' + c_x
new_y = y' + c_y


old_x = x - c_x
old_y = y - c_x

x = old_x*cosa + old_y*sina
y = -old_x*sina + old_y*cosa 


# Inversive mapping with bilinear interpolation 

1. Create new empty matrix, so turned image fits in it.
2. For each pixel in new matrix we perform inverse mapping to determine its color in original image.

3. For every result of inversive mapping we perform bilinear interpolation to determine its color. 
    x0,y0 - the top left corner
    x1,y1 - the bottom right corner
    x0 = round(x)
    x1 = x0 + 1
    y0 = round(y)
    y1 = y0 + 1

    adjacent pixels
    Q00 = I(x0, y0)
    Q10 = I(x1, y0)
    Q01 = I(x0, y1)
    Q11 = I(x1, y1)

    dx = x - x0 (0 <= dx < 1)
    dy = y - y0 (0 <= dy < 1)

    Calculate weight of each adjecent pixel:
    Q00_w = (1 - dx)(1 - dy)
    Q10_w = dx(1 - dy)
    Q01_w = (1 - dx)dy
    Q11_w = dx*dy

    Formule of bilenear interpolation 
    I(x, y) = Q00_w + Q10_w + Q01_w + Q11_w

In [166]:
# # Размер холста
# cos_a, sin_a = np.cos(np.deg2rad(78)), np.sin(np.deg2rad(78))
# new_w = math.ceil(abs(width * cos_a) + abs(height * sin_a))
# new_h = math.ceil(abs(height * cos_a) + abs(width * sin_a))

# # Перенос центра
# cy_new, cx_new = new_h // 2, new_w // 2
# cy_old, cx_old = height // 2, width // 2

# Y0 = Y - cy_new
# X0 = X - cx_new

# # Inverse mapping
# X_old = X0 * cos_a + Y0 * sin_a + cx_old
# Y_old = -X0 * sin_a + Y0 * cos_a + cy_old

# # Bilinear interpolation
# x0, y0 = np.floor(X_old).astype('uint8'), np.floor(Y_old).astype('uint8')

# x0 = np.clip(x0, 0, width-1)
# y0 = np.clip(y0, 0, height-1)
# x1 = np.clip(x0 + 1, 0, width-1)
# y1 = np.clip(y0 + 1, 0, height-1)


# dx = X_old - x0
# dy = Y_old - y0

# w00 = (1-dx)*(1-dy)
# w01 = dx*(1-dy)
# w10 = (1-dx)*dy
# w11 = dx*dy

# out = (
#     new_image[y0,x0] * w00[..., None] +
#     new_image[y0,x1] * w01[..., None] +
#     new_image[y1,x0] * w10[..., None] +
#     new_image[y1,x1] * w11[..., None]
#     )

# mask = (X_old>=0)&(X_old<width)&(Y_old>=0)&(Y_old<height)
# out[~mask] = 0

# turned_box = np.zeros((new_h, new_w, 3), dtype='uint8')
# turned_box[mask] = out[mask].astype('uint8')
# new_title = "turned_box_02_05"

Размытие (Распределение Гауса)
    Вход: картинка (h * w * c), радиус размытия: default = 2 pixels
    - написать функцию суммы соседей в 2 пикселя

In [167]:
# Определяем наши чистые цвета
R = [255, 0, 0]
G = [0, 255, 0]
B = [0, 0, 255]

# Создаем повторяющийся паттерн цветов: R, G, B, R, G, B...
colors = np.array([R, G, B], dtype='uint8')

size = 50
# Создаем сетку индексов (0, 1, 2, 0, 1, 2...)
# (i + j) % 3 создает диагональный сдвиг паттерна
rows, cols = np.indices((size, size))
pattern_indices = (rows + cols) % 3

# Теперь используем Fancy Indexing, чтобы расставить цвета
# pattern_indices имеет форму (50, 50), а colors - (3, 3)
# Результат будет (50, 50, 3)
new_image = colors[pattern_indices]
new_title = "before blur"


In [176]:
# Пока просто найдем среднее всех пикселей по радиусу
def channels_sum(r, x, y):
    height, width = (y-r, y+r+1), (x-r, x+r+1)
    shape = height[1] - height[0], width[1] - width[0]
    return height, width, shape
channels_sum(2, 0, 0)



((-2, 3), (-2, 3), (5, 5))

In [ ]:
# Creating Output image
plt.figure()
plt.imshow(new_image)
plt.title(new_title)
plt.axis('off')

plt.savefig(f"{output_dir}/{new_title}.png", dpi = 300)
plt.close()